In [8]:
import sys
import time
from qudi.util.network import netobtain
import importlib
sys.path.append('../xq1i')
import libxq1i
importlib.reload(libxq1i)

confocal2 = libxq1i.xq1i(pulsed_master_logic, pulsed_measurement_logic, sequence_generator_logic)

In [ ]:
freq_ranges = {
    0: [1340.0e6, 0.50e6, 50],
    1: [1340.0e6, 0.50e6, 50],
    2: [1340.0e6, 0.50e6, 50],
    3: [1340.0e6, 0.50e6, 50]
}
poi_list = [
    {'poi': 'NV_01', 'orientations': [0,1,2,3]},
    {'poi': 'NV_02', 'orientations': [0,1,2,3]},
]


for entry in poi_list:
    currPoi = entry['poi']
    print(f'go to POI {currPoi} ...')
    poi_manager_logic.go_to_poi(currPoi)
    poi_manager_logic.set_active_poi(currPoi)
    time.sleep(1)

    # switch laser on (for refocus)
    confocal2.pulsed_laser_on()
    # perform refocus
    scanner_gui.toggle_optimize(True)
    while scanner_gui.module_state() == 'locked' or (scanning_optimize_logic._sequence_index < len(scanning_optimize_logic._scan_sequence)):
        time.sleep(0.1)
    print( '... position and counts at peak after refocus: ' + scanner_gui.optimizer_dockwidget.result_label.text() )
    # switch laser off (refocus finished)
    pulsed_master_logic.toggle_pulsed_measurement(False)

    for orientation in entry['orientations']:
        print(f'start measurements for orientation {orientation} ...')
        freq_range = freq_ranges[orientation]
        
        # pulsed ODMR
        confocal2.pulsedODMR_params['freq_start'] = freq_range[0]
        confocal2.pulsedODMR_params['freq_step'] = freq_range[1]
        confocal2.pulsedODMR_params['num_of_points'] = freq_range[2]
        confocal2.pulsedODMR_sweeps = 60e3
        confocal2.do_pulsedODMR( f'{currPoi}_ornt_{orientation}')
        result_dict = netobtain( pulsed_measurement_logic.do_fit('Lorentzian Dip') )
        confocal2.calib_params['res_freq'] = result_dict.params['center'].value

        # Rabi
        confocal2.rabi_sweeps = 60e3
        confocal2.do_rabi( f'{currPoi}_ornt_{orientation}' )
        result_dict = netobtain( pulsed_measurement_logic.do_fit('Sine') )
        confocal2.generate_params['rabi_period'] = float(1 / (result_dict.params['frequency'].value))
        pulsed_master_logic.set_generation_parameters(confocal2.generate_params)

        # Hahn echo
        confocal2.hahn_sweeps = 60e3
        confocal2.do_echo( f'{currPoi}_ornt_{orientation}' )